In [1]:
from Define_Paulis import I, X, Y, Z, cd, c, n, Mdot, bkt
import scipy.linalg as ln
import numpy as np

def dgr(M):
    return np.conjugate(np.transpose(M))

We know what the single particle translation operator looks like:
\begin{equation}
T_{\text{single}}(4) = \begin{pmatrix} 0 & 0 & 0 & 1 \\ 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0  \end{pmatrix}
\end{equation}

The many body operator can then be found using the Thouless translation
$$
T(N) = e^{\sum_{ij}(\ln T_{\text{single}})_{ij} c^{\dagger}_ic_j}
$$

In [2]:
def T_single(N):
    t = 0*np.identity(N)
    for i in range(0,N):
        t[i,np.mod(i-1,N)] = 1
    return t

def T(N):
    logT = ln.logm(T_single(N))
    phi = 0*I(N)
    for i in range(N):
        for j in range(N):
            phi = phi + logT[i][j]*Mdot([cd(i,N),c(j,N)])
    return ln.expm(phi)

N=4
print('normalized:')
print( np.amax(np.abs( Mdot([dgr(T_single(N)),T_single(N)]) - np.identity(N) )) )
print( np.amax(np.abs( Mdot([dgr(T(N)),T(N)]) - I(N) )) )
print()

print('Translates cd and c:')
print( np.amax(np.abs( Mdot([T(N),cd(1,N),dgr(T(N))]) - cd(2,N) )) )
print( np.amax(np.abs( Mdot([T(N),c(1,N),dgr(T(N))]) - c(2,N) )) )
print( np.amax(np.abs( Mdot([T(N),cd(0,N),dgr(T(N))]) - cd(1,N) )) )
print( np.amax(np.abs( Mdot([T(N),cd(N-1,N),dgr(T(N))]) - cd(0,N) )) )
print( np.amax(np.abs( Mdot([T(N),n(N-1,N),dgr(T(N))]) - n(0,N) )) )
print()

psi0 = [0 for i in range(2**N)]
psi0[0] = 1
print('Translates psi')
print( np.amax(np.abs( Mdot([T(N),psi0]) - psi0 )) )
print( np.amax(np.abs( Mdot([T(N),cd(1,N),psi0]) - Mdot([cd(2,N),psi0]) )) )
print( np.amax(np.abs( Mdot([T(N),cd(0,N),cd(2,N),psi0]) - Mdot([cd(1,N),cd(3,N),psi0]) )) )

normalized:
0.0
3.1086244689504383e-15

Translates cd and c:
2.501374596100256e-15
2.501374596100256e-15
2.5877758945392665e-15
2.8934665993866104e-15
3.1086244689504383e-15

Translates psi
2.220446049250313e-16
2.037397855853302e-15
2.376958388704457e-15


The question I am interested in is if we find the eigenstates of the single particle operator 
$$ |k> = \sum_i f_{ki} |i> $$
and we use those to build creation operators
$$ a^{\dagger}_k = f_{ki}c^{\dagger}_i $$
can we generate the many body eigenstates with Slater determinants
$$ T a^{\dagger}_k a^{\dagger}_{k'} |0> = (k + k') a^{\dagger}_k a^{\dagger}_{k'} |0> ?$$
and if so, does this imply that $H$ is block diagonal in this many body basis?

In [3]:
def f(N):
    phi,y = ln.eig(T_single(N))
    arg = np.argsort( 1j*np.log(phi) )
    fn = np.transpose(y)
    phi = [phi[arg[n]] for n in range(N)]
    fn = [np.abs(fn[arg[n]][0])/fn[arg[n]][0] *fn[arg[n]] for n in range(N)]
    return fn

def phi(N):
    phin,y = ln.eig(T_single(N))
    arg = np.argsort(1j*np.log(phin)) 
    fn = dgr(y)
    phin = [phin[arg[n]] for n in range(N)]
    fn = [fn[arg[n]] for n in range(N)]
    return phin


def ad(n,N):
    F = f(N)
    out = 0*cd(0,N)
    for i in range(N):
        #print(F[n][i])
        out = out + F[n][i]*cd(i,N)
    return out



In [4]:
N = 4
psi0 = [0 for i in range(2**N)]
psi0[0] = 1


In [5]:
print('Single particle')
print( np.amax(np.abs( Mdot([T_single(N),f(N)[0]]) - phi(N)[0]*f(N)[0] )) )
print( np.amax(np.abs( Mdot([T_single(N),f(N)[1]]) - phi(N)[1]*f(N)[1] )) )
print( np.amax(np.abs( Mdot([T_single(N),f(N)[2]]) - phi(N)[2]*f(N)[2] )) )
print( np.amax(np.abs( Mdot([T_single(N),f(N)[3]]) - phi(N)[3]*f(N)[3] )) )

Single particle
6.661338147750939e-16
7.226859497166928e-16
2.7755575615628914e-16
7.226859497166928e-16


In [6]:
print('Many Body T single particle subspace')
print( np.amax(np.abs( Mdot([T(N),ad(0,N),psi0]) - phi(N)[0]*Mdot([ad(0,N),psi0]) )) )
print( np.amax(np.abs( Mdot([T(N),ad(1,N),psi0]) - phi(N)[1]*Mdot([ad(1,N),psi0]) )) )
print( np.amax(np.abs( Mdot([T(N),ad(2,N),psi0]) - phi(N)[2]*Mdot([ad(2,N),psi0]) )) )
print( np.amax(np.abs( Mdot([T(N),ad(3,N),psi0]) - phi(N)[3]*Mdot([ad(3,N),psi0]) )) )

Many Body T single particle subspace
1.531556499417396e-15
1.7407249664330279e-15
1.1122824539168496e-15
1.7038524493107786e-15


In [7]:
print('Many Body T two particle subspace')
print(  np.amax(np.abs( Mdot([T(N),ad(1,N),ad(0,N),psi0]) - phi(N)[1]*phi(N)[0]*Mdot([ad(1,N),ad(0,N),psi0]) )) )
print(  np.amax(np.abs( Mdot([T(N),ad(2,N),ad(0,N),psi0]) - phi(N)[2]*phi(N)[0]*Mdot([ad(2,N),ad(0,N),psi0]) )) )
print(  np.amax(np.abs( Mdot([T(N),ad(3,N),ad(0,N),psi0]) - phi(N)[0]*phi(N)[3]*Mdot([ad(3,N),ad(0,N),psi0]) )) )
print(  np.amax(np.abs( Mdot([T(N),ad(1,N),ad(2,N),psi0]) - phi(N)[1]*phi(N)[2]*Mdot([ad(1,N),ad(2,N),psi0]) )) )
print(  np.amax(np.abs( Mdot([T(N),ad(1,N),ad(3,N),psi0]) - phi(N)[1]*phi(N)[3]*Mdot([ad(1,N),ad(3,N),psi0]) )) )
print(  np.amax(np.abs( Mdot([T(N),ad(3,N),ad(2,N),psi0]) - phi(N)[3]*phi(N)[2]*Mdot([ad(3,N),ad(2,N),psi0]) )) )

Many Body T two particle subspace
1.1215077876880563e-15
1.4099419585860534e-15
2.764360880687203e-15
1.6337732681760999e-15
1.557421899916321e-15
1.238329836106091e-15


In [8]:
print('Many Body T three particle subspace')
print(  np.amax(np.abs( Mdot([T(N),ad(2,N),ad(1,N),ad(0,N),psi0]) - phi(N)[2]*phi(N)[1]*phi(N)[0]*Mdot([ad(2,N),ad(1,N),ad(0,N),psi0]) )) )
print(  np.amax(np.abs( Mdot([T(N),ad(2,N),ad(1,N),ad(3,N),psi0]) - phi(N)[2]*phi(N)[1]*phi(N)[3]*Mdot([ad(2,N),ad(1,N),ad(3,N),psi0]) )) )
print(  np.amax(np.abs( Mdot([T(N),ad(2,N),ad(0,N),ad(3,N),psi0]) - phi(N)[2]*phi(N)[0]*phi(N)[3]*Mdot([ad(2,N),ad(0,N),ad(3,N),psi0]) )) )
print(  np.amax(np.abs( Mdot([T(N),ad(1,N),ad(0,N),ad(3,N),psi0]) - phi(N)[1]*phi(N)[0]*phi(N)[3]*Mdot([ad(1,N),ad(0,N),ad(3,N),psi0]) )) )




Many Body T three particle subspace
8.427316352953737e-16
5.092714773960294e-16
2.6408673257912432e-15
2.38498770010664e-15


In [9]:
print('Many Body T four particle subspace')
print(  np.amax(np.abs( Mdot([T(N),ad(3,N),ad(2,N),ad(1,N),ad(0,N),psi0]) - phi(N)[3]*phi(N)[2]*phi(N)[1]*phi(N)[0]*Mdot([ad(3,N),ad(2,N),ad(1,N),ad(0,N),psi0]) )) )


Many Body T four particle subspace
3.0303191209571234e-15


Since $[H,T] = 0$ we know that H cannot change an eigenvector of T into one with a different eigenvalue. However, for many-body states, there can be degenerate eigenvalues. 

In [10]:
print('single particle eigenvalues')
print( '0:', np.round( phi(N)[0] ,3) )
print( '1:', np.round( phi(N)[1] ,3) )
print( '2:', np.round( phi(N)[2] ,3) )
print( '3:', np.round( phi(N)[3] ,3) )

print('two particle eigenvalues')
print( '01:', np.round( phi(N)[0]*phi(N)[1]  ,3) )
print( '02:', np.round( phi(N)[0]*phi(N)[2]  ,3) )
print( '03:', np.round( phi(N)[0]*phi(N)[3]  ,3) )
print( '12:', np.round( phi(N)[1]*phi(N)[2]  ,3) )
print( '13:', np.round( phi(N)[1]*phi(N)[3]  ,3) )
print( '23:', np.round( phi(N)[2]*phi(N)[3]  ,3) )
print( '01 = 23 and 02 = 13')

print('three particle eigenvalues')
print( '012:', np.round( phi(N)[0]*phi(N)[1]*phi(N)[2]  ,3) )
print( '013:', np.round( phi(N)[0]*phi(N)[1]*phi(N)[3]  ,3) )
print( '023:', np.round( phi(N)[0]*phi(N)[2]*phi(N)[3]  ,3) )
print( '123:', np.round( phi(N)[1]*phi(N)[2]*phi(N)[3]  ,3) )

single particle eigenvalues
0: (-1+0j)
1: 1j
2: (1+0j)
3: -1j
two particle eigenvalues
01: (-0-1j)
02: (-1+0j)
03: (-0+1j)
12: 1j
13: (1+0j)
23: -1j
01 = 23 and 02 = 13
three particle eigenvalues
012: (-0-1j)
013: (-1+0j)
023: (-0+1j)
123: (1+0j)


Now let us check how this works with the Hamiltonian

In [11]:
def neighbors(N):
    nbrs = []
    for i in range(N-1):
        nbrs.append([i,i+1])
        nbrs.append([i+1,i])
    nbrs.append([N-1,0])
    nbrs.append([0,N-1])
    return nbrs

def K(k,N):
    nbrs = neighbors(N)
    Kout = 0*I(N)
    for pair in nbrs:
        i = pair[0]
        j = pair[1]
        Kout = Kout + k*Mdot([cd(i,N),c(j,N)])
    return Kout
 
def D(d,N):
    Dout = 0*I(N)
    for i in range(0,N):
        j = np.mod(i+1,N)
        Dout = Dout + Mdot([n(i,N),n(j,N)])
    return d*Dout

def H(k,d,N):
    return K(k,N) + D(d,N) 




In [12]:
print('commutes:', np.amax(np.abs( Mdot([H(1,0.7,4),T(4)]) - Mdot([T(4),H(1,0.7,4)]) )) )

commutes: 3.0178014525816443e-15


In [13]:
h = H(1,0.7,4) 
psi01 = Mdot([ad(0,N),ad(1,N),psi0])
psi02 = Mdot([ad(0,N),ad(2,N),psi0])
psi03 = Mdot([ad(0,N),ad(3,N),psi0])
psi12 = Mdot([ad(1,N),ad(2,N),psi0])
psi13 = Mdot([ad(1,N),ad(3,N),psi0])
psi23 = Mdot([ad(2,N),ad(3,N),psi0])

psi2p = [psi01,psi02,psi03,psi12,psi13,psi23]

test = [ [ bkt(psi2p[i],h,psi2p[j]) for i in range(6)] for j in range(6)]

np.round( test ,5)

array([[-1.65-0.j,  0.  -0.j,  0.  -0.j,  0.  +0.j, -0.  -0.j, -0.35-0.j],
       [ 0.  +0.j,  0.7 +0.j,  0.  -0.j,  0.  -0.j,  0.  -0.j, -0.  -0.j],
       [ 0.  +0.j,  0.  +0.j, -1.65+0.j,  0.35-0.j,  0.  -0.j,  0.  +0.j],
       [ 0.  +0.j,  0.  +0.j,  0.35-0.j,  2.35-0.j, -0.  -0.j,  0.  -0.j],
       [-0.  +0.j,  0.  +0.j,  0.  +0.j, -0.  +0.j,  0.7 +0.j, -0.  -0.j],
       [-0.35-0.j, -0.  +0.j, -0.  +0.j,  0.  +0.j, -0.  +0.j,  2.35+0.j]])

As expected, the blocks are exectly those which have denenerate eigenvalues of T.  

## Rewrite H in terms of a

In [14]:
N = 4

fd = dgr( f(4) )

print( np.amax(np.abs(   fd[0][0]*ad(0,N) + fd[0][1]*ad(1,N) + fd[0][2]*ad(2,N) + fd[0][3]*ad(3,N)  - cd(0,N)  )))
print( np.amax(np.abs(   fd[1][0]*ad(0,N) + fd[1][1]*ad(1,N) + fd[1][2]*ad(2,N) + fd[1][3]*ad(3,N)  - cd(1,N)  )))
print( np.amax(np.abs(   fd[2][0]*ad(0,N) + fd[2][1]*ad(1,N) + fd[2][2]*ad(2,N) + fd[2][3]*ad(3,N)  - cd(2,N)  )))
print( np.amax(np.abs(   fd[3][0]*ad(0,N) + fd[3][1]*ad(1,N) + fd[3][2]*ad(2,N) + fd[3][3]*ad(3,N)  - cd(3,N)  )))

5.125906774580067e-16
8.326672684688674e-16
3.3782125623021557e-16
8.881784197001252e-16


In [15]:
f(N)[1]

array([ 5.00000000e-01-3.85185989e-34j, -1.37002935e-16-5.00000000e-01j,
       -5.00000000e-01-2.61083821e-16j,  2.85804968e-18+5.00000000e-01j])

In [16]:
print(np.amax(np.abs( f(N)[0] - [1/2*np.exp(-1j*np.pi*j) for j in range(N)] )))
print(np.amax(np.abs( f(N)[1] - [1/2*np.exp(-1j*np.pi/2*j) for j in range(N)] )))
print(np.amax(np.abs( f(N)[2] - [1/2*np.exp(1j*0*j) for j in range(N)] )))
print(np.amax(np.abs( f(N)[3] - [1/2*np.exp(1j*np.pi/2*j) for j in range(N)] )))

3.3306690738754696e-16
4.540754989429367e-16
2.220446049250313e-16
4.540754989429367e-16


In [17]:
print(np.amax(np.abs( fd[0] - [1/2*np.exp(1j*0*j) for j in range(N)] )))
print(np.amax(np.abs( fd[1] + [1/2*np.exp(-1j*np.pi/2*j) for j in range(N)] )))
print(np.amax(np.abs( fd[2] - [1/2*np.exp(1j*np.pi*j) for j in range(N)] )))
print(np.amax(np.abs( fd[3] + [1/2*np.exp(1j*np.pi/2*j) for j in range(N)] )))

3.3306690738754696e-16
4.041120497397176e-16
4.482315075900194e-16
4.529177942200601e-16


In [18]:
print( np.amax(np.abs(   ad(0,N) + ad(1,N) + ad(2,N) + ad(3,N)  - 2*cd(0,N)  )))
print( np.amax(np.abs(   -ad(0,N) - np.exp(-1j*np.pi/2)*ad(1,N) - np.exp(-1j*np.pi/2*2)*ad(2,N) - np.exp(-1j*np.pi/2*3)*ad(3,N)  - 2*cd(1,N)  )))
print( np.amax(np.abs(   ad(0,N) + np.exp(1j*np.pi)*ad(1,N) + np.exp(1j*np.pi*2)*ad(2,N) + np.exp(1j*np.pi*3)*ad(3,N)  - 2*cd(2,N)  )))
print( np.amax(np.abs(   -ad(0,N) - np.exp(1j*np.pi/2)*ad(1,N) - np.exp(1j*np.pi/2*2)*ad(2,N) - np.exp(1j*np.pi/2*3)*ad(3,N)  - 2*cd(3,N)  )))


6.921144475284663e-16
1.0010751633254106e-15
5.143910172419463e-16
9.069760197110526e-16


We should have that $$ c^{\dagger}_j = \sum_n e^{i \frac{j n \pi}{N}} $$
but because of the ordering something is a little off.  Let me just set $a^{\dagger}$ and $a$ by hand instead of getting them from the eigenstates of T.

In [19]:
def ad(n,N):
    out = 0*cd(0,N)
    for j in range(N):
        out = out + 1/np.sqrt(N)*np.exp(1j*j*n*2*np.pi/N)*cd(j,N)
    return out

def a(n,N):
    return dgr(ad(n,N))

In [20]:
psin = Mdot([ad(3,N),ad(1,N),psi0])
np.amax(np.abs( Mdot([T(N),psin]) - bkt(psin,T(N),psin)*psin ))

3.85210819184729e-16

In [21]:
h = H(1,0.7,4) 
psi01 = Mdot([ad(0,N),ad(1,N),psi0])
psi02 = Mdot([ad(0,N),ad(2,N),psi0])
psi03 = Mdot([ad(0,N),ad(3,N),psi0])
psi12 = Mdot([ad(1,N),ad(2,N),psi0])
psi13 = Mdot([ad(1,N),ad(3,N),psi0])
psi23 = Mdot([ad(2,N),ad(3,N),psi0])

psi2p = [psi01,psi02,psi03,psi12,psi13,psi23]

test = [ [ bkt(psi2p[i],h,psi2p[j]) for i in range(6)] for j in range(6)]

np.round( test ,5)

array([[ 2.35-0.j, -0.  -0.j, -0.  -0.j, -0.  +0.j, -0.  +0.j, -0.35-0.j],
       [-0.  +0.j,  0.7 +0.j,  0.  +0.j,  0.  -0.j,  0.  -0.j, -0.  +0.j],
       [-0.  +0.j,  0.  -0.j,  2.35-0.j,  0.35+0.j, -0.  -0.j, -0.  -0.j],
       [-0.  -0.j, -0.  -0.j,  0.35-0.j, -1.65+0.j,  0.  +0.j,  0.  -0.j],
       [-0.  -0.j,  0.  +0.j, -0.  +0.j,  0.  -0.j,  0.7 +0.j, -0.  -0.j],
       [-0.35+0.j, -0.  -0.j,  0.  +0.j, -0.  +0.j, -0.  +0.j, -1.65+0.j]])

In [22]:
print( np.amax(np.abs(   ad(0,N) + ad(1,N) + ad(2,N) + ad(3,N)  - 2*cd(0,N)  )))
print( np.amax(np.abs(   ad(0,N) + np.exp(-1j*np.pi/2)*ad(1,N) + np.exp(-1j*np.pi/2*2)*ad(2,N) + np.exp(-1j*np.pi/2*3)*ad(3,N)  - 2*cd(1,N)  )))
print( np.amax(np.abs(   ad(0,N) + np.exp(1j*np.pi)*ad(1,N) + np.exp(1j*np.pi*2)*ad(2,N) + np.exp(1j*np.pi*3)*ad(3,N)  - 2*cd(2,N)  )))
print( np.amax(np.abs(   ad(0,N) + np.exp(1j*np.pi/2)*ad(1,N) + np.exp(1j*np.pi/2*2)*ad(2,N) + np.exp(1j*np.pi/2*3)*ad(3,N)  - 2*cd(3,N)  )))


2.3409673969408664e-16
1.4409059796375077e-16
7.347880794884119e-16
7.347880794884119e-16


Okay now we are set.  Let's write $K$ and $D$ in terms of $a^{\dagger}$ and $a$.

In [23]:
#Let's take a look at what states are in these eigenstates of T.
psin = Mdot([ad(1,N),ad(2,N),psi0])
for i in range(len(psin)):
    if np.abs(psin[i]) > 0.0001:
        print(i,psin[i],bin(i))

3 (0.25+0.24999999999999997j) 0b11
5 (-0.5+9.184850993605148e-17j) 0b101
6 (0.24999999999999992-0.2500000000000001j) 0b110
9 (0.24999999999999994-0.2500000000000001j) 0b1001
10 (1.8369701987210297e-16+0.5j) 0b1010
12 (-0.2500000000000001-0.2499999999999999j) 0b1100


We know that $K$ is diagonal in this basis:
$$ K = \sum_n 2k\cos(n \frac{2\pi}{N}) a^{\dagger}_n a_n $$

In [25]:
def Kn(k,N):
    Ktst = 0*I(N)
    for n in range(N):
        Ktst = Ktst + 2*k*np.cos(n*2*np.pi/N)*Mdot([ad(n,N),a(n,N)])
    return Ktst

k=0.67
np.amax(np.abs( Kn(k,N) - K(k,N) ))

2.46154006628618e-16

On the other hand, $D$ is only block diagonal,

$$ D = \frac{d}{N}\sum_{nn'mm'} e^{i \frac{2\pi}{N}(m-m')} a^{\dagger}_{n}a_{n'}a^{\dagger}_{m}a_{m'} \delta_{0,n-n'+m-m'}$$

So we have the condition that $D$ only connects modes with the property $\text{mod}(n-n'+m-m',N) = 0$.  For $N=4$ there are only two off diagonal elements: $$0+3 = 1+2$$ and $$ \text{mod(2+3,N)} = 0+1 $$

So for N=4,
$$ D = \frac{d}{4} (\sum_{mm'}a^{\dagger}_ma_m a^{\dagger}_{m'}a_{m'} + a^{\dagger}_0a^{\dagger}_2a_3a_1 + a^{\dagger}_0a^{\dagger}_1a_2a_3 + h.c.) $$




The number of possible terms in $D$ is less than $N^4$ so it is subexponential.  We can find the terms by checking all possible combinations of $n,n',m,m'$ which is again $N\times N\times N \times N = N^4$.

In [76]:
def find_terms(N):
    out = []
    for n in range(N):
        for nn in range(N):
            for m in range(N):
                for mm in range(N):
                    if np.mod(n-nn+m-mm,N) == 0:
                        if n != m and nn != mm:
                            if n != nn:
                                if n != mm:
                                    in_list = 0
                                    for old in out:
                                        if n == old[0] and m == old[1] and nn == old[2] and mm == old[3]:
                                            in_list = 1
                                        if n == old[1] and m == old[0] and nn == old[2] and mm == old[3]:
                                            in_list = 1
                                        if n == old[0] and m == old[1] and nn == old[3] and mm == old[2]:
                                            in_list = 1
                                        if n == old[1] and m == old[0] and nn == old[3] and mm == old[2]:
                                            in_list = 1
                                        if nn == old[0] and mm == old[1] and n == old[2] and m == old[3]:
                                            in_list = 1
                                        if nn == old[1] and mm == old[0] and n == old[2] and m == old[3]:
                                            in_list = 1
                                        if nn == old[0] and mm == old[1] and n == old[3] and m == old[2]:
                                            in_list = 1
                                        if nn == old[1] and mm == old[0] and n == old[3] and m == old[2]:
                                            in_list = 1
                                    if in_list == 0:
                                        out.append([n,m,nn,mm])
    return out

find_terms(6)

[[0, 3, 1, 2],
 [0, 4, 1, 3],
 [0, 5, 1, 4],
 [0, 1, 2, 5],
 [0, 5, 2, 3],
 [0, 1, 3, 4],
 [0, 2, 3, 5],
 [0, 3, 4, 5],
 [1, 4, 2, 3],
 [1, 5, 2, 4],
 [1, 2, 4, 5],
 [2, 5, 3, 4]]

In general, $D$ connects states with particles in 
\begin{eqnarray}
 &[0,N-1] &\leftrightarrow [1,N-2] &\leftrightarrow [2,N-3] &\ldots [N-1,1] &\leftrightarrow [0,N-1]   \\
 &[0,N-2] &\leftrightarrow [1,N-3] &\leftrightarrow [2,N-4] &\ldots [N-1,N-1] &\leftrightarrow [0,N-2] \\
 &[0,N-3] &\leftrightarrow [1,N-4] &\leftrightarrow [2,N-5] &\ldots [N-1,N-2] &\leftrightarrow [0,N-3] \\
  &  &  & \vdots & &  \\
 &[0,1] &\leftrightarrow [1,0] &\leftrightarrow [2,N-1] &\ldots [N-1,2] &\leftrightarrow [0,1] \\
 &[0,0] &\leftrightarrow [1,N-1] &\leftrightarrow [2,N-2] &\ldots [N-1,1] &\leftrightarrow [0,0] 
\end{eqnarray}

In [44]:
def Du(d,N):
    Dtst = 0*I(N)
    for n in range(N):
        for nn in range(N):
            for m in range(N):
                for mm in range(N):
                    if np.mod(n-nn+m-mm,N) == 0:
                        Dtst = Dtst + d/N*np.exp(1j*2*np.pi*(m-mm)/N)*Mdot([cd(n,N),c(nn,N),cd(m,N),c(mm,N)])
    return Dtst

In [32]:
def Dn(d,N):
    Dtst = 0*I(N)
    for n in range(N):
        for nn in range(N):
            for m in range(N):
                for mm in range(N):
                    if np.mod(n-nn+m-mm,N) == 0:
                        Dtst = Dtst + d/N*np.exp(1j*2*np.pi*(m-mm)/N)*Mdot([ad(n,N),a(nn,N),ad(m,N),a(mm,N)])
    return Dtst

In [33]:
from Define_Paulis import n as num

def D(d,N):
    Dout = 0*I(N)
    for i in range(0,N):
        j = np.mod(i+1,N)
        Dout = Dout + Mdot([num(i,N),num(j,N)])
    return d*Dout

d=0.4
np.amax(np.abs( Dn(d,N) - D(d,N) ))

2.220446049250313e-16